# CS211 Final Project

In [9]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

heights = pd.read_csv('https://raw.githubusercontent.com/alexdowns0/CS211_Final_Project/main/Heights.csv')

In [20]:
## Cache the sorted ages, because we will use them a lot.
height_lower = 0
height_upper = 100
sorted_heights = heights['Height'].clip(lower=height_lower, upper=height_upper).sort_values()

def min_height():
    clipped_heights = heights['Height'].clip(lower=0, upper=100)
    return clipped_heights.min()

def ls_min():
    return max(sorted_heights.iloc[0] - height_lower, sorted_heights.iloc[1]  - sorted_heights.iloc[0]) #new min - old min 

print('Actual minimum height:', min_height(), 'inches')
print('Local sensitivity of the minimum:', ls_min(), 'inches')



Actual minimum height: 65.0 inches
Local sensitivity of the minimum: 65.0 inches
91.0


In [ ]:
# more chunks means less noise


def f(chunk):
    return (chunk['Height'].quantile(q=1))

def saa_min_age(k, epsilon):
    df = heights['Height']
  
    # Calculate the number of rows in each chunk
    chunk_size = int(np.ceil(df.shape[0] / k))
    
    # Step 1: split `df` into chunks
    xs = [df[i:i+chunk_size] for i in range(0,df.shape[0],chunk_size)]
    
    # Step 2: run f on each x_i and clip its output
    answers = [f(x_i) for x_i in xs]
    
    u = 95
    l = 60
    clipped_answers = np.clip(answers, l, u)
    

    # Step 3: take the noisy mean of the clipped answers
    #noisy_mean = laplace_mech(np.mean(clipped_answers), (u-l)/k, epsilon)    equivalent to the 2 lines below
    noisy_sum = laplace_mech(np.sum(clipped_answers), u-1, epsilon)
    noisy_mean = noisy_sum / k
    return noisy_mean



saa_min_age(500, 1.0)